In [8]:
# Iric
iric_path = '/home/ebutz/ESL2024/data/full_iric/iric.csv'
mapped_iric_path  = '/home/ebutz/ESL2024/data/full_iric/altailed_mapped_iric.pickle'
altails_dict_path = '/home/ebutz/ESL2024/data/full_iric/altail_iric_DICT.pickle'
check_dicts = True

In [4]:
print("\nImports...")

import torch

import sys
sys.path.append('/home/ebutz/ESL2024/code/utils' )
from play_with_complex import *
from data_utils import *
from train_utils import *
from model_utils import *
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import optuna
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.sampler import NegativeSampling
import torch_geometric.transforms as T

import pickle
import os
import sys


import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

from nxontology.imports import from_file


Imports...


/home/ebutz/miniconda3/envs/pyg2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
{'homogeneous': False, 'scorelist_size': 1000, 'split_ratio': 0.8, 'val_ratio': 0.1, 'test_ratio': 0.1,
 'num_neighbors': [70, 55, 13, 89, 85], 'batch_size': 1024, 'train_neg_sampling_ratio': 224, 'epochs': 18,
 'disjoint_train_ratio': 0.6, 'lr': 0.0015308253347932983, 'stopper_metric': 'mrr', 'stopper_direction': 'maximize',
 'stopper_patience': 5, 'stopper_frequency': 1, 'stopper_relative_delta': 0.05, 'gamma': 1.3, 'alpha': 0.42680473078813763, 'gnn_layer': 'ResGatedGraphConv',
 'dropout': 0.1, 'norm': 'DiffGroupNorm', 'aggregation': 'min', 'hidden_channels': 115, 'num_layers': 3, 'attention_heads': 4}

{'homogeneous': False,
 'scorelist_size': 1000,
 'split_ratio': 0.8,
 'val_ratio': 0.1,
 'test_ratio': 0.1,
 'num_neighbors': [70, 55, 13, 89, 85],
 'batch_size': 1024,
 'train_neg_sampling_ratio': 224,
 'epochs': 18,
 'disjoint_train_ratio': 0.6,
 'lr': 0.0015308253347932983,
 'stopper_metric': 'mrr',
 'stopper_direction': 'maximize',
 'stopper_patience': 5,
 'stopper_frequency': 1,
 'stopper_relative_delta': 0.05,
 'gamma': 1.3,
 'alpha': 0.42680473078813763,
 'gnn_layer': 'ResGatedGraphConv',
 'dropout': 0.1,
 'norm': 'DiffGroupNorm',
 'aggregation': 'min',
 'hidden_channels': 115,
 'num_layers': 3,
 'attention_heads': 4}

In [6]:
import wandb

# config = {'val_ratio': 0.1,'homogeneous': False, 'scorelist_size': 1000, 'split_ratio': 0.8,'val_ratio': 0.1, 'test_ratio': 0.1, 'num_neighbors': [70, 55, 13, 89, 85],
#           'batch_size': 1024, 'train_neg_sampling_ratio': 224, 'epochs': 18, 'disjoint_train_ratio': 0.6,
#           'lr': 0.0015308253347932983, 'stopper_metric': 'mrr', 'stopper_direction': 'maximize',
#           'stopper_patience': 5, 'stopper_frequency': 1, 'stopper_relative_delta': 0.05, 'gamma': 1.3,
#           'alpha': 0.42680473078813763, 'gnn_layer': 'ResGatedGraphConv', 'dropout': 0.1,
#           'norm': 'DiffGroupNorm', 'aggregation': 'min', 'hidden_channels': 115, 'num_layers': 3, 'attention_heads': 4}

# run = wandb.init(config = config)
run = wandb.init()

config = wandb.config

config.val_ratio=0.1
config.homogeneous=False
config.scorelist_size=1000
config.split_ratio=0.8
config.val_ratio=0.1
config.test_ratio=0.1
config.num_neighbors=[70,55,13,89,85]
config.batch_size=1024
config.train_neg_sampling_ratio=224
config.epochs=18
config.disjoint_train_ratio=0.6
config.lr=0.0015308253347932983
config.stopper_metric= 'mrr'
config.stopper_direction="maximize"
config.stopper_patience=5
config.stopper_frequency=1
config.stopper_relative_delta=0.05
config.gamma=1.3
config.alpha=0.42680473078813763
config.gnn_layer='ResGatedGraphConv'
config.dropout=0.1
config.norm='DiffGroupNorm'
config.aggregation = 'min'
config.hidden_channels=115
config.num_layers=3
config.attention_heads=4
config.homogeneous = False
config.labels = {'head' : 'genes', 'relation' : 'gene_ontology', 'tail' : 'go'}

print(type(wandb.config))
print(config.homogeneous)
print(config.val_ratio)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: butzelliot (esl2024). Use `wandb login --relogin` to force relogin


<class 'wandb.sdk.wandb_config.Config'>
False
0.1


# Loading datas

In [7]:
pd.read_csv(iric_path)


/tmp/ipykernel_9504/2580875774.py:1: DtypeWarning: Columns (1,2,3,4,6,7,8,9,10,11,12,13,14,15,18,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(iric_path)


,Unnamed: 0,definition,is_a,name,namespace,Annotation score,Gene Ontology,Genomic Sequence,InterPro:description,PANTHER,...,Gene Name Synonyms,Prosite_patterns,Trait Class,Trait Ontology,ncoils,Keyword,Family,PRINTS,Plant Ontology,Allele
0,GO:0000001,"The distribution of mitochondria, including th...",GO:0048311|GO:0048308,mitochondrion inheritance,biological_process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GO:0000002,The maintenance of the structure and integrity...,GO:0007005,mitochondrial genome maintenance,biological_process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GO:0000003,The production of new individuals that contain...,GO:0008150,reproduction,biological_process,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GO:0000006,Enables the transfer of zinc ions (Zn2+) from ...,GO:0005385,high-affinity zinc transmembrane transporter a...,molecular_function,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GO:0000007,Enables the transfer of a solute or solutes fr...,GO:0005385,low-affinity zinc ion transmembrane transporte...,molecular_function,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118388,OsNippo12g261100,NaN,NaN,NaN,NaN,NaN,NaN,GCTAGCCGGCGCCGCCCCGACCGTTGCCGTCGCTCGCCGTGTGCTC...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118389,OsNippo12g261150,NaN,NaN,NaN,NaN,NaN,NaN,ACTAGCTTATCAGGGGAGCCTTGATGCCGAGAGACATGTGGATTCG...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118390,OsNippo12g261200,NaN,NaN,NaN,NaN,NaN,NaN,AATGACGGCCACGACGACGACAGGAACGGCGCCTGGATCCGCCGTC...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118391,OsNippo12g261600,NaN,NaN,NaN,NaN,NaN,NaN,GCTAGCTGTTACAAGGTTTGTTGGGTAGCCTAGCTACACAAGAAAG...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data = load_iric_data('/home/ebutz/ESL2024/data/full_iric/iric.csv', featureless=False)
data  = T.ToUndirected(merge=True)(data) # Convert the graph to an undirected graph. Creates reverse edges for each edge.
data = T.RemoveDuplicatedEdges()(data) # Remove duplicated edges
print(data)
assert data.validate()

train_data, val_data, test_data = split_data(data, config)
train_loader, val_loader, test_loader = build_dataloaders(train_data, val_data, test_data, config)

/home/ebutz/ESL2024/code/utils/train_utils.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_links['source_node'] = df_links.index


HeteroData(
  genes={ x=[72335, 29] },
  go={ x=[42979, 3] },
  po={ x=[1662, 2] },
  traito={ x=[1636, 1] },
  prosite_profiles={ x=[627, 1] },
  prosite_patterns={ x=[652, 1] },
  superfamily={ x=[957, 1] },
  panther={ x=[5971, 1] },
  prints={ x=[420, 1] },
  (genes, interacts_with, genes)={ edge_index=[2, 1120849] },
  (genes, gene_ontology, go)={ edge_index=[2, 169248] },
  (genes, trait_ontology, traito)={ edge_index=[2, 15080] },
  (genes, plant_ontology, po)={ edge_index=[2, 6125] },
  (genes, profile, prosite_profiles)={ edge_index=[2, 12239] },
  (genes, pattern, prosite_patterns)={ edge_index=[2, 6677] },
  (genes, family, superfamily)={ edge_index=[2, 20742] },
  (genes, panther_id, panther)={ edge_index=[2, 25319] },
  (genes, prints_id, prints)={ edge_index=[2, 4365] },
  (go, is_a, go)={ edge_index=[2, 136436] },
  (po, is_a, po)={ edge_index=[2, 3548] },
  (traito, is_a, traito)={ edge_index=[2, 3770] },
  (go, rev_gene_ontology, genes)={ edge_index=[2, 169248] },
  (t

In [11]:
batchy = next(iter(train_loader))
print(batchy)

HeteroData(
  genes={
    x=[25072, 29],
    n_id=[25072],
    num_sampled_nodes=[6],
    src_index=[1024],
  },
  go={
    x=[42976, 3],
    n_id=[42976],
    num_sampled_nodes=[6],
    dst_pos_index=[1024],
    dst_neg_index=[1024, 224],
  },
  po={
    x=[1474, 2],
    n_id=[1474],
    num_sampled_nodes=[6],
  },
  traito={
    x=[1580, 1],
    n_id=[1580],
    num_sampled_nodes=[6],
  },
  prosite_profiles={
    x=[624, 1],
    n_id=[624],
    num_sampled_nodes=[6],
  },
  prosite_patterns={
    x=[652, 1],
    n_id=[652],
    num_sampled_nodes=[6],
  },
  superfamily={
    x=[952, 1],
    n_id=[952],
    num_sampled_nodes=[6],
  },
  panther={
    x=[5512, 1],
    n_id=[5512],
    num_sampled_nodes=[6],
  },
  prints={
    x=[420, 1],
    n_id=[420],
    num_sampled_nodes=[6],
  },
  (genes, interacts_with, genes)={
    edge_index=[2, 486698],
    e_id=[486698],
    num_sampled_edges=[5],
  },
  (genes, gene_ontology, go)={
    edge_index=[2, 32472],
    edge_label=[81240],
    e_

In [ ]:
# # Model to train :
# hidden_channels = 176
# batch_size      = 4096
# epochs          = 200
# eval_period     = 2
# lin_factor      = 1

# use_wandb  = True

# params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{epochs}_epochs_{batch_size}_BS_on_full_iric"
# model_parameters_path = "/home/ebutz/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name

# # Ontology
# ontology_path = "/home/ebutz/ESL2024/data/go-basic.json.gz"

# # ------------- Cuda ------------- #

# print("\nCuda check...")

# device = 'cpu'

# # ------------- Loading datas ------------- #

# iric = load_iric_data('data/iric.csv', featureless=False)
# display(iric)

# print("\nLoading iric...")
# mapped_iric = pd.read_pickle(mapped_iric_path)
# print(mapped_iric.head())
# print('mapped_alt_tails type :', type(mapped_iric.iloc[0]['mapped_alt_tails']))

# GO_to_map = mapped_iric.set_index('object')['mapped_object'].to_dict()
# map_to_GO = {value: key for key, value in GO_to_map.items()}

# if check_dicts:
#     looks_ok: bool = True
#     for i in tqdm(range(len(list(mapped_iric['object']))), desc = "Checking GO to MAP dict"):
#         if GO_to_map[mapped_iric['object'][i]]!=mapped_iric['mapped_object'][i] :
#             looks_ok = False
#     print('GO - Mapping dicts looks ok :', looks_ok)

# with open(altails_dict_path, 'rb') as handle:
#     mapped_alt_tails = pickle.load(handle)
# print("Alternative tails dict (first key-value pair):", list(mapped_alt_tails.items())[0])

# # ------------- Loading ontology ------------- #

# print("\nLoading ontology...")
# nxo = from_file(ontology_path)
# nxo.freeze()
# pwc.nxo = nxo

# # ------------- Making global variables accessibles to pwc ------------- #

# pwc.map_to_GO        = map_to_GO
# pwc.mapped_alt_tails = mapped_alt_tails
# pwc.device           = device

# # ------------- Making datasets ------------- #


# print("\nDataset looks valid :",hetero_iric.validate(raise_on_error=True))

# print('All done.')

# Init model

In [ ]:
gnn_layers = get_gnn_layers(config)
norm_layers = get_norm_layers(config, len(data.node_types))

model = Model(config, data, norm_layers, gnn_layers).to(device)

# Test and train

In [ ]:
! pip install torch-sparse

  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [113 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-310
      creating build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/__init__.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/add.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/bandwidth.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/cat.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/coalesce.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/convert.py -> build/lib.linux-x86_64-cpython-310/torch_sparse
      copying torch_sparse/diag

In [ ]:
# conda list
! conda list --explicit

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
@EXPLICIT
https://conda.anaconda.org/conda-forge/linux-64/_libgcc_mutex-0.1-conda_forge.tar.bz2
https://repo.anaconda.com/pkgs/main/noarch/_sysroot_linux-64_curr_repodata_hack-3-haa98f57_10.tar.bz2
https://repo.anaconda.com/pkgs/main/linux-64/blas-1.0-openblas.conda
https://repo.anaconda.com/pkgs/main/linux-64/ca-certificates-2024.3.11-h06a4308_0.conda
https://conda.anaconda.org/anaconda/noarch/font-ttf-dejavu-sans-mono-2.37-hd3eb1b0_0.tar.bz2
https://conda.anaconda.org/anaconda/noarch/font-ttf-inconsolata-2.001-hcb22688_0.tar.bz2
https://conda.anaconda.org/anaconda/noarch/font-ttf-source-code-pro-2.030-hd3eb1b0_0.tar.bz2
https://conda.anaconda.org/anaconda/noarch/font-ttf-ubuntu-0.83-h8b1ccd4_0.tar.bz2
https://conda.anaconda.org/conda-forge/linux-64/ld_impl_linux-64-2.40-h41732ed_0.conda
https://conda.anaconda.org/conda-forge/noarch/libgcc-devel_linux-64-13.2.0

In [ ]:
conda install https://repo.anaconda.com/pkgs/main/linux-64/urllib3-2.1.0-py310h06a4308_1.conda



Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [ ]:

# ----------------- Loops ----------------- #
@timer_func
def evaluate(config, loader, model, criterion, compute_all_metrics=False, loader_type='validation', stopper_metric=False):
    """
    Evaluate the model on a given data loader.

    Parameters
    ----------
    config : object
        An object containing the configuration parameters for the model.    
    loader : DataLoader
        The data loader to evaluate the model on.
    model : Model
        The model to evaluate.
    criterion : callable
        The loss function to use for evaluation.
    compute_all_metrics : bool, optional
        Whether to compute all metrics or partially. Default is False.
    loader_type : str, optional
        The type of the loader ('validation' or 'test'). Default is 'validation'.
    stopper_metric : str or bool, optional
        The metric that dictates when to stop training. If False, only the loss is computed. Default is False.

    Returns
    -------
    tuple
        A tuple of the evaluation metric and the loss, depending on the value of `stopper_metric`.
        If compute_all_metrics is True, the function logs all metrics to W&B.
    """
    model.eval()
    num_neg_samples = loader.neg_sampling.amount
    with torch.no_grad():
        if stopper_metric or compute_all_metrics:
            ground_truths = torch.tensor([], device='cpu')
            preds = torch.tensor([], device='cpu')
            indexes = torch.tensor([], device='cpu')
            total_loss, total_examples = 0, 0
            index_end = loader.batch_size

        for sampled_data in loader:
            sampled_data = sampled_data.to(device)

            batch_size = len(sampled_data[config.labels['tail']].dst_pos_index)
            pred = model(sampled_data, config)
            pos_samples = torch.ones(batch_size, device=device)
            neg_samples = torch.zeros(num_neg_samples*batch_size, device=device)
            ground_truth = torch.cat((pos_samples, neg_samples))

            if stopper_metric or compute_all_metrics: # Store preds and truths for all batches, and compute indices to calc metrics
                index_pos = torch.arange(end=index_end, start=index_end-batch_size) # index for predictions with pos. ground_truth
                index_neg = torch.arange(end=index_end, start=index_end-batch_size).repeat_interleave(num_neg_samples)
                index = torch.cat((index_pos, index_neg))
                indexes = torch.cat((indexes, index.to('cpu')))
                preds = torch.cat((preds, pred.to('cpu')))
                ground_truths = torch.cat((ground_truths, ground_truth.to('cpu')))
                index_end += batch_size

            eval_loss = criterion(pred, ground_truth, gamma=config.gamma, alpha=config.alpha)

            # if not stopper_metric: # Just logging loss
            #     wandb.log({"running_val_loss": eval_loss})
            #     break
            # else:
            total_loss += float(eval_loss) * pred.numel() 
            total_examples += pred.numel()
            eval_loss = total_loss / total_examples

            # if stopper_metric and not compute_all_metrics:
            #     if index_end >= 2048: # Compute approx. intermediate metric on a few datapoints to speed up hyperopt process
            #         break 

        model.train()

        if stopper_metric and not compute_all_metrics: # Compute only the metric that dictates stopping
            indexes = indexes.long()

            # # Log heatmap between prediction and ground truth. Useful for visualization / debugging. overhead 0.2s per epoch for a single sample.
            # heatmap = heatmaps(preds, ground_truths, indexes) 
            # wandb.log({f"heatmap": wandb.Image(heatmap)})
            # heatmap.close() # Free up memory

            match stopper_metric:
                case "val_loss":
                    eval_loss = total_loss / total_examples
                    return eval_loss
                
                case "mrr":
                    mrr = RetrievalMRR().to(device)
                    return mrr(preds, ground_truths, indexes=indexes), eval_loss
                
                case "hit_at_10":
                    hit_at_10 = RetrievalHitRate(top_k=10).to(device)
                    return hit_at_10(preds, ground_truths, indexes=indexes), eval_loss
                
                case "hit_at_5":
                    hit_at_5 = RetrievalHitRate(top_k=5).to(device)
                    return hit_at_5(preds, ground_truths, indexes=indexes), eval_loss
                
                case "hit_at_3":
                    hit_at_3 = RetrievalHitRate(top_k=3).to(device)
                    return hit_at_3(preds, ground_truths, indexes=indexes), eval_loss
                
                case "hit_at_1":
                    hit_at_1 = RetrievalHitRate(top_k=1).to(device)
                    return hit_at_1(preds, ground_truths, indexes=indexes), eval_loss
                
                case _:
                    raise ValueError(f"Unrecognized stopper metric: '{stopper_metric}'")
                
        if compute_all_metrics: # Compute all metrics at the end of training
            indexes = indexes.long()
            mrr = RetrievalMRR().to(device)
            hit_at_10 = RetrievalHitRate(top_k=10).to(device)
            hit_at_5 = RetrievalHitRate(top_k=5).to(device)
            hit_at_3 = RetrievalHitRate(top_k=3).to(device)
            hit_at_1 = RetrievalHitRate(top_k=1).to(device)
            
            mrr = mrr(preds, ground_truths, indexes=indexes)
            hit_at_10 = hit_at_10(preds, ground_truths, indexes=indexes)
            hit_at_5 = hit_at_5(preds, ground_truths, indexes=indexes)
            hit_at_3 = hit_at_3(preds, ground_truths, indexes=indexes)
            hit_at_1 = hit_at_1(preds, ground_truths, indexes=indexes)

            wandb.log({
                f"{loader_type}MRR": mrr, f"{loader_type}hit_at_10": hit_at_10, f"{loader_type}hit_at_5": hit_at_5, f"{loader_type}hit_at_3": hit_at_3, f"{loader_type}hit_at_1": hit_at_1
            })


# Set device
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = Model(config, data, norm_layers, gnn_layers).to(device)
print(model)

criterion = sigmoid_focal_loss
optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
# optimizer = Lion(model.parameters(), lr=config.lr, weight_decay=1e-2)
early_stopper = EarlyStopper(frequency=config.stopper_frequency, patience=config.stopper_patience, direction=config.stopper_direction, relative_delta=config.stopper_relative_delta)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

@timer_func
def train(config, train_loader, val_loader, model, criterion, optimizer, early_stopper):
    for epoch in range(config.epochs):
        total_loss = total_examples = 0

        for sampled_data in tqdm(train_loader, desc="Training"):

            sampled_data = sampled_data.to(device)
            pred = model(sampled_data, config)
            print(sampled_data)

            pos_samples = torch.ones(len(sampled_data[config.labels['tail']].dst_pos_index), device=device)
            neg_samples = torch.zeros(len(sampled_data[config.labels['tail']].dst_neg_index.view(-1)), device=device) # As many zeroes as there are negative samples * batch_size
            ground_truth = torch.cat((pos_samples, neg_samples))
            # Add gaussian noise to pos_samples to simulate noisy labels. Added noise must be negative and not exceed 1.
            # ground_truth += torch.normal(mean=0, std=1, size=(len(ground_truth),), device=device)
            # apply sigmoid
            ground_truth = torch.sigmoid(ground_truth)
            # ground_truth = torch.sigmoid(ground_truth)
            loss = criterion(pred, ground_truth, gamma=config.gamma, alpha=config.alpha)
            wandb.log({"loss": loss})
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            total_loss += float(loss) * pred.numel()
            total_examples += pred.numel()
        train_loss = total_loss / total_examples
        print(f"Epoch: {epoch:03d}, Avg. Loss: {train_loss:.10f}")
        # scheduler.step()

        # Compute metrics and check for early stopping
        score, val_loss = evaluate(config, val_loader, model, criterion, stopper_metric=config.stopper_metric)
        wandb.log({"avg_loss": train_loss, "val_loss": val_loss, f"{config.stopper_metric}": score})

        # early_stopper(score)
        # if early_stopper.early_stop:
        #     print("Early stopping triggered at epoch", epoch)
        #     break

    print("Training done.")

train(config, train_loader, val_loader, model, criterion, optimizer, early_stopper)
evaluate(config, val_loader, model, criterion, compute_all_metrics=True, loader_type='validation', stopper_metric=False)
evaluate(config, test_loader, model, criterion, compute_all_metrics=True, loader_type='test', stopper_metric=False)

Device: 'cuda'
Model(
  (encoder): GraphModule(
    (convs): ModuleList(
      (0-2): 3 x ModuleDict(
        (genes__interacts_with__genes): ResGatedGraphConv((-1, -1), 115)
        (genes__gene_ontology__go): ResGatedGraphConv((-1, -1), 115)
        (genes__trait_ontology__traito): ResGatedGraphConv((-1, -1), 115)
        (genes__plant_ontology__po): ResGatedGraphConv((-1, -1), 115)
        (genes__profile__prosite_profiles): ResGatedGraphConv((-1, -1), 115)
        (genes__pattern__prosite_patterns): ResGatedGraphConv((-1, -1), 115)
        (genes__family__superfamily): ResGatedGraphConv((-1, -1), 115)
        (genes__panther_id__panther): ResGatedGraphConv((-1, -1), 115)
        (genes__prints_id__prints): ResGatedGraphConv((-1, -1), 115)
        (go__is_a__go): ResGatedGraphConv((-1, -1), 115)
        (po__is_a__po): ResGatedGraphConv((-1, -1), 115)
        (traito__is_a__traito): ResGatedGraphConv((-1, -1), 115)
        (go__rev_gene_ontology__genes): ResGatedGraphConv((-1, -1), 

Training:   0%|          | 0/80 [00:00<?, ?it/s]


ImportError: Caught ImportError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/ebutz/miniconda3/envs/pyg/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/ebutz/miniconda3/envs/pyg/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/home/ebutz/miniconda3/envs/pyg/lib/python3.10/site-packages/torch_geometric/loader/link_loader.py", line 211, in collate_fn
    out = self.link_sampler.sample_from_edges(
  File "/home/ebutz/miniconda3/envs/pyg/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py", line 334, in sample_from_edges
    out = edge_sample(inputs, self._sample, self.num_nodes, self.disjoint,
  File "/home/ebutz/miniconda3/envs/pyg/lib/python3.10/site-packages/torch_geometric/sampler/neighbor_sampler.py", line 666, in edge_sample
    out = sample_fn(seed_dict, se